In [1]:

from itertools import permutations
from joblib import Parallel, delayed
from more_itertools import distinct_permutations
from tqdm import tqdm
import numpy as np
import pickle

from qulacs import QuantumCircuit

from mcr.equiv_check import (
    equiv,
    pauli_bit_equivalence_check,
    equivalence_check_via_mqt_qcec,
)

from mcr.gate_apply import (
    PauliBit,
    grouping,
    loop_optimization,
    set_clifford_to_qulacs,
    zhang_optimization,
)

from opt_using_mcr import test_algorithm
from optimizer import (
    full_optimization,
    optimize_data_loop,
    mcr_swap,
    have_common_pauli_str,
    attempt_mcr_retry,
    three_layer_nontrivial_swap,
)

In [2]:
filetype = "seq"  # "small" or "seq"
nqubits = 3  # Number of qubits in the circuit
# with open(f"unopt_{filetype}.pickle", "rb") as f:
with open(f"unopt_{nqubits}.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi/4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi/4))
data.append(PauliBit("Z" * nqubits, -np.pi / 4))  # Add identity gate

In [3]:
def load_data_from_pickle(filepath: str, nqubits: int) -> list:
    """pickle形式のファイルを読み込む

    Args:
        filepath (str): pickle形式のファイルパス

    Returns:
        list: 読み込んだデータのリスト
    """
    with open(filepath, mode="br") as f:
        data = pickle.load(f)
    pauli_bit_sequences = []
    for elem in data:
        initial_pauli_str, theta, qubit_indices = elem
        pauli_str = ["I"] * nqubits
        for pauli, qubit_idx in zip(initial_pauli_str, qubit_indices):
            pauli_str[qubit_idx] = pauli
        pauli_bit = PauliBit("".join(pauli_str), theta)
        pauli_bit_sequences.append(pauli_bit)
    return pauli_bit_sequences
nqubits = 6
# data = load_data_from_pickle("te_pai_data.pickle", nqubits=nqubits)

In [6]:
clifford, non_clifford = full_optimization(data, max_iter=5, show_opt_log=True)

🔁 Optimization iteration: 1 / 5
🎉 Optimization success in iteration 1: 120 → 42
🎉 Optimization success in iteration 2: 42 → 30
🎉 Optimization success in iteration 3: 30 → 16
🎉 Optimization success in iteration 4: 16 → 14
🔍 No optimization in iteration 5: 14 → 14
⚙️  Additional optimization: 1 / 5
🎉 Optimization success in iteration 1: 22 → 16
🎉 Optimization success in iteration 2: 16 → 14
🔍 No optimization in iteration 3: 14 → 14
🔁 Optimization iteration: 2 / 5
🔍 No optimization in iteration 1: 14 → 14
⚙️  Additional optimization: 2 / 5
🎉 Optimization success in iteration 1: 22 → 18
🎉 Optimization success in iteration 2: 18 → 14
🔍 No optimization in iteration 3: 14 → 14
🔁 Optimization iteration: 3 / 5
🎉 Optimization success: 14 → 0


In [5]:
len(non_clifford)

0

In [6]:
new_clifford_lst, new_optimized_data = optimize_data_loop(
    non_clifford, show_opt_log=True, max_attempts=1
)
len(new_optimized_data)

🔍 No optimization in iteration 1: 14 → 14


14

In [7]:
m = grouping(non_clifford)

In [8]:
m

[[ZXZYXXIZIZXYXXYXXZII, 0.785],
 [YZIIIYYXZYYIYIXXYZXY, -0.785, YZYXYZZIIYXYIZXZYZZX, -0.785],
 [YZIZXIIIYXYZXYIIZXIZ, 0.785,
  ZIYYIXYYXZZXZXXIIZZY, 0.785,
  ZIYYIZZYYYYYIIYIYYXZ, 0.785],
 [YXYIIXIIZYZZYXIXIZIY, -0.785],
 [IYIXYYZIZIYXYYXYYIZZ, 0.785, XXXYIYZIXIYZIYIYIXXX, 0.785],
 [YXYIIXIIZYZZYXIXIZIY, 0.785, ZIXIZIIIIXYXZYXIXIYX, -0.785],
 [IIYYZZZIYZZXXXXZXYZY, -0.785, IYIXYYZIZIYXYYXYYIZZ, -0.785],
 [YZYXYZZIIYXYIZXZYZZX, 0.785, ZIZYIZZIYYXIYZIZIYXZ, 0.785],
 [IYIXYYZIZIYXYYXYYIZZ, -0.785],
 [YXIXYIXXIYIXIYIZIZYX, 0.785, YXIYZIYIYIXXIXYYZIXX, -0.785],
 [IYZZXXIYXZYYXIYYXYIX, 0.785, ZIIIZYXYZIIIYIIZXXII, -0.785],
 [YXIYZIYIYIXXIXYYZIXX, 0.785, ZZZZZZZZZZZZZZZZZZZZ, -0.785]]

In [9]:
attempt_mcr_retry(non_clifford)

[[ZXZYXXIZIZXYXXYXXZII, 0.785,
  ZXXZZIXYZZYIZYYZXZYZ, 0.785,
  ZXXZZIXYZZYIZYYZXZYZ, -0.785],
 [YZIIIYYXZYYIYIXXYZXY, -0.785, YZYXYZZIIYXYIZXZYZZX, -0.785],
 [YZIZXIIIYXYZXYIIZXIZ, 0.785,
  ZIYYIXYYXZZXZXXIIZZY, 0.785,
  ZIYYIZZYYYYYIIYIYYXZ, 0.785],
 [YXYIIXIIZYZZYXIXIZIY, -0.785,
  ZYZZYXIIXYZXIXXXYYYI, 0.785,
  ZYZZYXIIXYZXIXXXYYYI, -0.785],
 [IYIXYYZIZIYXYYXYYIZZ, 0.785, XXXYIYZIXIYZIYIYIXXX, 0.785],
 [YXYIIXIIZYZZYXIXIZIY, 0.785, ZIXIZIIIIXYXZYXIXIYX, -0.785],
 [IIYYZZZIYZZXXXXZXYZY, -0.785, IYIXYYZIZIYXYYXYYIZZ, -0.785],
 [YZYXYZZIIYXYIZXZYZZX, 0.785, ZIZYIZZIYYXIYZIZIYXZ, 0.785],
 [IYIXYYZIZIYXYYXYYIZZ, -0.785,
  IYIYZYIXXYZXYXZZXZIZ, 0.785,
  IYIYZYIXXYZXYXZZXZIZ, -0.785],
 [YXIXYIXXIYIXIYIZIZYX, 0.785, YXIYZIYIYIXXIXYYZIXX, -0.785],
 [IYZZXXIYXZYYXIYYXYIX, 0.785, ZIIIZYXYZIIIYIIZXXII, -0.785],
 [YXIYZIYIYIXXIXYYZIXX, 0.785, ZZZZZZZZZZZZZZZZZZZZ, -0.785]]

In [10]:
c, d = full_optimization(non_clifford)
len(d)

22

In [11]:
m = grouping(non_clifford)
m[4] += [
    PauliBit("IIYYZZZIYZZXXXXZXYZY", np.pi / 4),
    PauliBit("IIYYZZZIYZZXXXXZXYZY", -np.pi / 4),
]
m
e,f = full_optimization(m)
len(e)

509

In [12]:
from optimizer import attempt_mcr_retry
# a = attempt_mcr_retry(non_clifford)
# len(sum(a, []))

In [13]:
len(full_optimization(m)[1])

0

In [14]:
# mcr_swap(a, with_mcr_index=True)

In [15]:
from perform_mcr import optimize_data_loop
a, b = optimize_data_loop(data)
len(b)

🎉 Optimization success in iteration 1: 4796 → 3170
🎉 Optimization success in iteration 2: 3170 → 1006
🎉 Optimization success in iteration 3: 1006 → 482
🎉 Optimization success in iteration 4: 482 → 210
🎉 Optimization success in iteration 5: 210 → 76
🎉 Optimization success in iteration 6: 76 → 0


0

In [14]:
# equiv([[],data],[clifford, non_clifford])

In [15]:
# itertools.permutations で全列挙は計算量が膨大なので、numpy配列でインデックスの順列を生成し、必要な部分だけ計算することで高速化します。
seq_a = [
    PauliBit("ix", -np.pi / 4),
    PauliBit("zi", -np.pi / 4),
    PauliBit("xy", np.pi / 4),
    PauliBit("zy", -np.pi / 4),
    PauliBit("yy", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
    PauliBit("ix", np.pi / 4),
    PauliBit("xi", np.pi / 4),
    PauliBit("yz", -np.pi / 4),
    PauliBit("xz", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
]
seq_b = [PauliBit("yx", -np.pi / 4)]
equiv([[], seq_a], [[],seq_b])

False

In [16]:
A = [
    PauliBit("IZ", -np.pi / 4),
    PauliBit("ZY", np.pi / 4),
    PauliBit("XY", -np.pi / 4),
    PauliBit("IZ", -np.pi / 4),
    PauliBit("XZ", -np.pi / 4),
    PauliBit("IY", -np.pi / 4),
    PauliBit("YZ", np.pi / 4),
    PauliBit("IX", -np.pi / 4),
    PauliBit("ZI", np.pi / 4),
    PauliBit("YY", -np.pi / 4),
    PauliBit("XI", -np.pi / 4),
    PauliBit("ZI", -np.pi / 4),
    PauliBit("ZZ", np.pi / 4),
    PauliBit("XI", np.pi / 4),
    PauliBit("ZX", np.pi / 4),
    PauliBit("XX", np.pi / 4),
]

In [17]:
m = grouping(A)
# m[8] += [PauliBit("XZ", -np.pi / 4)]
# m.insert(8, [PauliBit("XZ", np.pi / 4)])
m[8] += [PauliBit("XZ", np.pi / 4), PauliBit("XZ", -np.pi / 4)]
# m.insert(8, [PauliBit("XZ", -np.pi / 4)])
m

[[IZ, -0.785],
 [ZY, 0.785],
 [XY, -0.785],
 [IZ, -0.785, XZ, -0.785],
 [IY, -0.785],
 [YZ, 0.785],
 [IX, -0.785, ZI, 0.785],
 [YY, -0.785],
 [XI, -0.785, XZ, 0.785, XZ, -0.785],
 [ZI, -0.785, ZZ, 0.785],
 [XI, 0.785],
 [ZX, 0.785],
 [XX, 0.785]]

In [18]:
# ini = m.copy()
seq = mcr_swap(m)
clifford_lst, optimized_data = optimize_data_loop(
    seq, show_opt_log=True, max_attempts=3
)


🎉 Optimization success in iteration 1: 19 → 18
🎉 Optimization success in iteration 2: 18 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16


In [19]:
clifford_lst, optimized_data = optimize_data_loop(
    seq, show_opt_log=True, max_attempts=3
)


🎉 Optimization success in iteration 1: 19 → 18
🎉 Optimization success in iteration 2: 18 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16
🔍 No optimization in iteration 3: 16 → 16


In [20]:
optimized_data

[IZ, -0.785,
 ZY, 0.785,
 XY, -0.785,
 IZ, -0.785,
 XZ, -0.785,
 IY, -0.785,
 YZ, 0.785,
 IX, -0.785,
 ZI, 0.785,
 YY, -0.785,
 XI, -0.785,
 ZI, -0.785,
 ZZ, 0.785,
 XI, 0.785,
 ZX, 0.785,
 XX, 0.785]

In [21]:
three_layer_nontrivial_swap(grouping(A),with_mcr_index=True)

find_nontrivial_swap called
3layer swap only
left_bits: XY, -0.785, right_bits: IY, -0.785
solution:  [[[IY, -0.785], [IZ, -0.785, XZ, -0.785], [XY, -0.785]]]
at index  2
After 3-layer swap: [IZ, -0.785, ZY, 0.785, IY, -0.785, IZ, -0.785, XZ, -0.785, XY, -0.785, YZ, 0.785, IX, -0.785, ZI, 0.785, YY, -0.785, XI, -0.785, ZI, -0.785, ZZ, 0.785, XI, 0.785, ZX, 0.785, XX, 0.785]


([IZ, -0.785,
  ZY, 0.785,
  IY, -0.785,
  IZ, -0.785,
  XZ, -0.785,
  XY, -0.785,
  YZ, 0.785,
  IX, -0.785,
  ZI, 0.785,
  YY, -0.785,
  XI, -0.785,
  ZI, -0.785,
  ZZ, 0.785,
  XI, 0.785,
  ZX, 0.785,
  XX, 0.785],
 {2})

In [22]:
p =grouping(optimize_data_loop(
    three_layer_nontrivial_swap(grouping(A)), show_opt_log=True
)[1])
p

find_nontrivial_swap called
3layer swap only
left_bits: XY, -0.785, right_bits: IY, -0.785
solution:  [[[IY, -0.785], [IZ, -0.785, XZ, -0.785], [XY, -0.785]]]
at index  2
After 3-layer swap: [IZ, -0.785, ZY, 0.785, IY, -0.785, IZ, -0.785, XZ, -0.785, XY, -0.785, YZ, 0.785, IX, -0.785, ZI, 0.785, YY, -0.785, XI, -0.785, ZI, -0.785, ZZ, 0.785, XI, 0.785, ZX, 0.785, XX, 0.785]
🔍 No optimization in iteration 1: 16 → 16


[[IZ, -0.785],
 [IY, -0.785, ZY, 0.785],
 [IZ, -0.785, XZ, -0.785],
 [IX, -0.785, ZI, 0.785],
 [XY, -0.785, YZ, 0.785],
 [YY, -0.785],
 [XI, -0.785],
 [ZI, -0.785, ZZ, 0.785],
 [XI, 0.785],
 [ZX, 0.785],
 [XX, 0.785]]

In [23]:
seq_a = [
    PauliBit("ix", -np.pi / 4),
    PauliBit("xy", np.pi / 4),
    PauliBit("zy", -np.pi / 4),
    PauliBit("yy", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
    PauliBit("ix", np.pi / 4),
    PauliBit("xi", np.pi / 4),
    PauliBit("yz", -np.pi / 4),
    PauliBit("xz", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
]

# seq_b = [
#     PauliBit("XZ", np.pi / 4),
#     PauliBit("ZI", -np.pi / 4),
#     PauliBit("ZZ", np.pi / 4),
#     PauliBit("XZ", -np.pi / 4),
# ]
# equiv([[], seq_a], [[], seq_b])

In [24]:
m = grouping(seq_a)
m

[[IX, -0.785],
 [XY, 0.785],
 [ZY, -0.785],
 [YY, -0.785, ZZ, -0.785],
 [IX, 0.785, XI, 0.785],
 [YZ, -0.785],
 [XZ, -0.785],
 [ZZ, -0.785]]

In [25]:

m[2] += [PauliBit("YZ", np.pi / 4), PauliBit("YZ", -np.pi / 4)]
# optimize_data_loop(mcr_swap(m))
mcr_swap(m)

[IX, -0.785,
 XY, 0.785,
 YZ, -0.785,
 ZY, -1.571,
 YY, -0.785,
 ZZ, -0.785,
 ZY, 1.571,
 IX, 0.785,
 XI, 0.785,
 ZY, -0.785,
 YZ, 0.785,
 YZ, -0.785,
 XZ, -0.785,
 ZZ, -0.785]